# Model 2: Modified Autoencoder (Reduced Latent Space)

This notebook implements a **modified autoencoder** with **reduced latent dimension** compared to Model 1.

**Key Modification:**
- **Latent Dimension:** Reduced from 64 to **32**

**Architecture:**
- **Encoder:** 784 → 256 → 128 → 64 → **32** (latent)
- **Decoder:** **32** (latent) → 64 → 128 → 256 → 784
- **Optimizer:** RMSprop (lr=0.001) - same as Model 1
- **Loss:** Mean Squared Error - same as Model 1
- **Training:** 20 epochs, batch_size=128 - same as Model 1

**Expected Results:**
- Higher reconstruction loss due to increased information compression
- More reconstruction artifacts (blurriness, loss of fine details)
- Demonstrates the **compression vs. quality trade-off**

In [1]:
# Import required libraries
import os
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from PIL import Image

## 1. Load and Preprocess Fashion-MNIST Dataset

In [2]:
# Load Fashion-MNIST dataset
print("Loading Fashion-MNIST dataset...")
(x_train, _), (x_test, _) = fashion_mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten input images for dense autoencoder
x_train_flat = x_train.reshape((-1, 28 * 28))
x_test_flat = x_test.reshape((-1, 28 * 28))

print(f"Training samples: {x_train_flat.shape[0]}")
print(f"Test samples: {x_test_flat.shape[0]}")

Loading Fashion-MNIST dataset...
Training samples: 60000
Test samples: 10000


## 2. Build Autoencoder Architecture (Reduced Latent Dimension)

**MODIFICATION:** Reduced latent dimension from 64 to 32. This forces the bottleneck to compress 784 pixels into only 32 dimensions, increasing information loss. The model must discard more details to fit the essential information into the smaller space, resulting in blurrier reconstructions with more artifacts.

In [3]:
# Build the autoencoder architecture (same as Model 1, but with reduced latent dimension)
input_dim = 28 * 28
# MODIFICATION: Reduced latent dimension from 64 to 32
# This forces the bottleneck to compress 784 pixels into only 32 dimensions,
# increasing information loss. The model must discard more details to fit
# the essential information into the smaller space, resulting in blurrier
# reconstructions with more artifacts.
latent_dim = 32  # Reduced from 64 in Model 1

# Encoder (architecture identical to Model 1)
encoder_input = Input(shape=(input_dim,), name='encoder_input')
x = Dense(256, activation='relu', name='enc_dense_1')(encoder_input)
x = Dense(128, activation='relu', name='enc_dense_2')(x)
x = Dense(64, activation='relu', name='enc_dense_3')(x)
latent = Dense(latent_dim, activation='relu', name='latent')(x)

encoder = Model(inputs=encoder_input, outputs=latent, name='encoder')

# Decoder (architecture identical to Model 1)
decoder_input = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(64, activation='relu', name='dec_dense_1')(decoder_input)
x = Dense(128, activation='relu', name='dec_dense_2')(x)
x = Dense(256, activation='relu', name='dec_dense_3')(x)
decoder_output = Dense(input_dim, activation='sigmoid', name='decoder_output')(x)

decoder = Model(inputs=decoder_input, outputs=decoder_output, name='decoder')

# Autoencoder: encoder + decoder
ae_input = encoder_input
ae_output = decoder(encoder(ae_input))
autoencoder = Model(inputs=ae_input, outputs=ae_output, name='autoencoder')

In [4]:
# Compile model (same optimizer and loss as Model 1)
optimizer = RMSprop(learning_rate=0.001)  # same as Model 1
loss_fn = MeanSquaredError()  # reconstruction loss
autoencoder.compile(optimizer=optimizer, loss=loss_fn)

## 4. Model Summaries

In [5]:
# Print model summaries
print("\n" + "=" * 50)
print("Encoder Summary:")
print("=" * 50)
encoder.summary()

print("\n" + "=" * 50)
print("Decoder Summary:")
print("=" * 50)
decoder.summary()

print("\n" + "=" * 50)
print("Autoencoder Summary:")
print("=" * 50)
autoencoder.summary()


Encoder Summary:


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_1 (Dense)             │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_2 (Dense)             │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_3 (Dense)             │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 32)             │         2,080 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 244,192 (953.88 KB)

 Trainable params: 244,192 (953.88 KB)

...

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 32)             │       244,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 784)            │       244,944 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 489,136 (1.87 MB)

 Trainable params: 489,136 (1.87 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the Model

In [6]:
# Train the model (same epochs and batch size as Model 1)
epochs = 20  # same as Model 1
batch_size = 128  # same as Model 1

print("\n" + "=" * 50)
print("Training Autoencoder...")
print("=" * 50)

history = autoencoder.fit(
    x_train_flat, x_train_flat,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(x_test_flat, x_test_flat),
    verbose=1
)


Training Autoencoder...


Epoch 1/20


  1/469 ━━━━━━━━━━━━━━━━━━━━ 2:44 352ms/step - loss: 0.1645

 23/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1688    

 46/469 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1666

...

412/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0187

429/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0187

448/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0187

468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0187

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0186 - val_loss: 0.0180


## 6. Training Loss per Epoch

In [7]:
# Print training loss per epoch
print("\n" + "=" * 50)
print("Training Loss per Epoch:")
print("=" * 50)
for epoch, loss in enumerate(history.history['loss'], start=1):
    print(f"Epoch {epoch}: {loss:.6f}")

# Final training loss
final_train_loss = history.history['loss'][-1]


Training Loss per Epoch:
Epoch 1: 0.069358
Epoch 2: 0.041361
Epoch 3: 0.036531
Epoch 4: 0.031148
Epoch 5: 0.028029
Epoch 6: 0.026414
Epoch 7: 0.025249
Epoch 8: 0.024323
Epoch 9: 0.023542
Epoch 10: 0.022891
Epoch 11: 0.022194
Epoch 12: 0.021579
Epoch 13: 0.021017
Epoch 14: 0.020579
Epoch 15: 0.020168
Epoch 16: 0.019832
Epoch 17: 0.019459
Epoch 18: 0.019148
Epoch 19: 0.018867
Epoch 20: 0.018615


## 7. Evaluate on Test Set

In [8]:
# Evaluate on test set
print("\n" + "=" * 50)
print("Evaluating on test set...")
print("=" * 50)
test_loss = autoencoder.evaluate(x_test_flat, x_test_flat, batch_size=batch_size, verbose=1)


Evaluating on test set...
 1/79 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0170

47/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0178

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0180


## 8. Reconstruct Test Images

Using the same 5 test image indices (0, 1, 2, 3, 4) as Model 1 for direct comparison.

In [9]:
# Reconstruct test images (using same indices as Model 1)
print("\n" + "=" * 50)
print("Reconstructing test images...")
print("=" * 50)

# Select exactly 5 test images (same indices as Model 1: 0, 1, 2, 3, 4)
n_images = 5
indices = [0, 1, 2, 3, 4]
test_samples = x_test_flat[indices]
reconstructed = autoencoder.predict(test_samples, verbose=0)

# Reshape for saving
originals = test_samples.reshape((n_images, 28, 28))
recos = reconstructed.reshape((n_images, 28, 28))

# Ensure output directory exists
output_dir = "reconstructions"
os.makedirs(output_dir, exist_ok=True)

print("\nSelected test image indices:")
for idx in indices:
    print(f"  Index: {idx}")

# Save images with Model 2 naming convention
# Note: We expect more reconstruction artifacts due to reduced latent space
print("\nSaving images...")
for i in range(n_images):
    # Save reconstructed images with Model 2 naming
    reco_path = os.path.join(output_dir, f"reconstructed_model2_{i+1}.png")
    
    # Reconstructed image (with expected artifacts from information compression)
    img2 = (np.clip(recos[i], 0, 1) * 255).astype('uint8')
    Image.fromarray(img2, mode='L').save(reco_path)
    
    print(f"  Saved: {reco_path}")


Reconstructing test images...

Selected test image indices:
  Index: 0
  Index: 1
  Index: 2
  Index: 3
  Index: 4

Saving images...
  Saved: reconstructions/reconstructed_model2_1.png
  Saved: reconstructions/reconstructed_model2_2.png
  Saved: reconstructions/reconstructed_model2_3.png
  Saved: reconstructions/reconstructed_model2_4.png
  Saved: reconstructions/reconstructed_model2_5.png


## 9. Final Results

In [10]:
# Print final model outputs
print("\n----------------------------------------------")
print("MODEL 2: MODIFIED AUTOENCODER (REDUCED LATENT SPACE)")
print("----------------------------------------------")
print(f"Latent Dimension Used: {latent_dim}")
print(f"Final Training Loss: {final_train_loss:.6f}")
print(f"Final Test Reconstruction Loss: {test_loss:.6f}")


----------------------------------------------
MODEL 2: MODIFIED AUTOENCODER (REDUCED LATENT SPACE)
----------------------------------------------
Latent Dimension Used: 32
Final Training Loss: 0.018615
Final Test Reconstruction Loss: 0.018021
